In [25]:
# import dependencies
import pandas as pd
import numpy as np
import warnings
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report

warnings.filterwarnings("ignore")

### Pre-processing

In [29]:
scotus_df = pd.read_csv("../Resources/SCOTUS_OHEncoded.csv", index_col=0)
df_106 = scotus_df[scotus_df["justice"] == 106]
df_106.tail()

,term_1953,term_1960,term_1962,term_1965,term_1968,term_1972,term_1973,term_1977,term_1984,term_1987,...,lawType_3,lawType_4,lawType_5,lawType_6,lawType_8,lawType_9,precedentAlteration_0,precedentAlteration_1,direction,justice
13801,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,2,106
13810,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,1,106
13819,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,2,106
13827,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,1,0,2,106
13836,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,1,0,1,106


In [30]:
target_106 = df_106["direction"]
target_names = ["conservative", "liberal"]

In [31]:
data_106 = df_106.drop(["direction", "justice"], axis=1)
data_106.head()

,term_1953,term_1960,term_1962,term_1965,term_1968,term_1972,term_1973,term_1977,term_1984,term_1987,...,lawType_1,lawType_2,lawType_3,lawType_4,lawType_5,lawType_6,lawType_8,lawType_9,precedentAlteration_0,precedentAlteration_1
0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,1,0
1,0,0,0,0,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,1,0
2,0,0,0,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,1,0
3,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,1,0
4,0,0,0,0,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,1,0


In [32]:
# dummy (One-Hot) encoding
# term_df = pd.get_dummies(data_106["term"], prefix="term")
# precAlt_df = pd.get_dummies(data_106["precedentAlteration"], prefix="precedentAlteration")
# issueArea_df = pd.get_dummies(data_106["issueArea"], prefix="issueArea")
# lawType_df = pd.get_dummies(data_106["lawType"], prefix="lawType")
# caseOriginState_df = pd.get_dummies(data_106["caseOriginState"], prefix="caseOriginState")

In [33]:
# concatenate encoded features into mega-dataframe
# oneHot_features = pd.concat([term_df, precAlt_df], axis=1)
# oneHot_features = pd.concat([oneHot_features, issueArea_df], axis=1)
# oneHot_features = pd.concat([oneHot_features, lawType_df], axis=1)
# oneHot_features = pd.concat([oneHot_features, caseOriginState_df], axis=1)
# oneHot_features.head()

In [35]:
# assign as X, y, and feature_names variables (to simplify replication)
X = data_106
y = target_106
feature_names = data_106.columns

### Evaluating different classifiers

In [36]:
clf_lr = LogisticRegression()
clf_rf = RandomForestClassifier()
clf_nb = GaussianNB()
clf_svm = SVC(kernel='linear', probability=True)

print('5-fold cross validation:\n')

labels = ["Logistic Regression", "Random Forest", "Naive Bayes", "Linear SVM"]

for clf, label in zip([clf_lr, clf_rf, clf_nb, clf_svm], labels):
    scores = cross_val_score(clf, X, y, cv=5, 
                             scoring='f1', n_jobs=-1)
    print("F1: %0.3f [%s]" % (scores.mean(), label))


5-fold cross validation:

F1: 0.616 [Logistic Regression]
F1: 0.593 [Random Forest]
F1: 0.111 [Naive Bayes]
F1: 0.654 [Linear SVM]


In [37]:
# using all but naive bayes (lowest score) for voting classifier estimators
v_clf_hard = VotingClassifier(estimators = [(labels[0], clf_lr),
                                            (labels[1], clf_rf),
                                            (labels[3], clf_svm)],
                                            voting = 'hard')

In [38]:
v_clf_soft = VotingClassifier(estimators = [(labels[0], clf_lr),
                                            (labels[1], clf_rf),
                                            (labels[3], clf_svm)],
                                            voting = 'soft')

In [39]:
new_labels = ["Logistic Regression", "Random Forest", "Linear SVM", 
              "Voting_Classifier_Hard", "Voting_Classifier_Soft"]

for (clf, label) in zip([clf_lr, clf_rf, clf_svm, v_clf_hard, v_clf_soft], new_labels):
    scores = cross_val_score(clf, X, y,cv=5,
                             scoring='f1', n_jobs=-1)
    print("F1: %0.3f [%s]" % (scores.mean(), label))

F1: 0.616 [Logistic Regression]
F1: 0.607 [Random Forest]
F1: 0.654 [Linear SVM]
F1: 0.638 [Voting_Classifier_Hard]
F1: 0.635 [Voting_Classifier_Soft]


### Training the SVM model
Linear SVM performed best of the classifiers evaluated for this justice's data

In [40]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [41]:
model = SVC(kernel='linear')
model.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [42]:
# Model Accuracy
print('Train Acc: %.3f' % model.score(X_train, y_train))
print('Test Acc: %.3f' % model.score(X_test, y_test))

Train Acc: 0.607
Test Acc: 0.628


In [43]:
predictions = model.predict(X_test)
print(classification_report(y_test, predictions,
                            target_names=target_names))

              precision    recall  f1-score   support

conservative       0.63      0.90      0.75       330
     liberal       0.59      0.21      0.31       218

   micro avg       0.63      0.63      0.63       548
   macro avg       0.61      0.56      0.53       548
weighted avg       0.62      0.63      0.57       548



In [44]:
print(confusion_matrix(predictions, y_test))
predictions[:15], y_test[:15].ravel()

[[298 172]
 [ 32  46]]


(array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1], dtype=int64),
 array([1, 1, 1, 2, 2, 2, 2, 1, 1, 1, 1, 2, 2, 1, 2], dtype=int64))

### Save Model and Confirm Load

In [45]:
#Save Model
#https://machinelearningmastery.com/save-load-machine-learning-models-python-scikit-learn/
import pickle

filename = '106_model.sav'
pickle.dump(model, open(filename, 'wb'))

In [46]:
#Load Model from Disk
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, y_test)
print(result)

0.6277372262773723
